# IceSat-2 Satellite Polar Ice Thickness
## Jeff Liu
#### Bayesian modeling of ice thickness given a satellite freeboard measurement with a hierarchical nonlinear multivariate truncated Gaussian model
#### Posterior sampled using Metropolis-within-Gibbs style Markov Chain Monte Carlo.

See the paper for full details of the model.

In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, multivariate_normal, invgamma
from math import sqrt

## Note on Constants

I got a (somewhat) arbitrary estimate of average snow depth of 0.15 meters from https://earthobservatory.nasa.gov/images/146758/mapping-snow-on-arctic-sea-ice.
I also borrowed average ice thickness of 2 meters from lecture, along with the lower/upper bounds that come up in the `CheckPriorBounds` function.

The variances I chose are wide to be less informative, but the bounds are truncated to prevent this from introducing inaccuracy.

In [26]:
# Constants
var_rw = 0
var_ri = 0
var_rs = 0
var_H = 0

mean_rw = 1020
mean_ri = 910
mean_rs = 300
mean_H = 2 # from lecture
mean_S = 0.15 # eyeballed estimate from CryoSat-2 website
mu_x = np.array([mean_rw, mean_ri, mean_rs, mean_H, mean_S]) # combined into one mean vector

# chosen somewhat arbitrarily
alpha_eps = 0
beta_eps = 0

alpha_s = 0
beta_s = 0

### Forward Model
Computes the model's prediction of Freeboard given the three densities, snow thickness, and ice thickness.

In [25]:
def ForwardModel(rw, ri, rs, H, S):
    return (rw-ri)/rw * (H - S*(rs/(rw-ri)))

### Likelihood Function

Evaluate log of the likelihood of the freeboard measurement `F` given params `x` and some hierarchally modeled additive error `var_eps`:

$f(F | x, \epsilon) \sim N(0, \epsilon) $

where the value to evaluate the likelihood at is $F - g(x)$ where $g(x)$ is the forward model.

In [29]:
def LogLikelihood(x, F, var_eps):
    """
    ARGS:
        x (np.array): A numpy array of length 5 containing the model parameters: rw, ri, rs, H, S
        F (float): A scalar containing the observation of freeboard F
        var_eps (float): A scalar containing the given error variance
        
    RETURNS:
        (float): A scalar containing log( f(F | x, var_eps) )
    """
    rw, ri, rs, H, S = x
    diff = F - forwardModel(rw, ri, rs, H, S)
    sigma_eps = sqrt(var_eps)
    
    return norm.logpdf(diff, 0, sigma_eps)

### Bounds checking on prior

Since our prior consists of truncated normal distributions, we need to check if the proposed parameters `x` fit this.

In [30]:
def CheckPriorBounds(x):
    rw, ri, rs, H, S = x
    return 1010 <= rw <= 1030 and 750 <= ri <= 940 and 100 <= rs <= 400 and 0.1 <= H <= 5 and S >= 0

### Prior Density of parameters

Evaluates log of the prior density of proposed parameters `x` given some variable snow variance:

$f(x|\sigma^2_S) \sim N(\mu_x, cov_x)$

where the covariance contains our snow variance, and where we evaluate the density with our proposed `x`.

Assume that the covariance matrix is diagonal; i.e. we have an independent prior.

In [31]:
def LogPrior(x, var_S):
    """
    ARGS:
        x (np.array): A numpy array of length 5 containing the model parameters: rw, ri, rs, H, S
        var_S (float): A scalar containing the snow variance
        
    RETURNS:
        (float): A scalar containing log( f(x | var_S) )
    """
    
    if not CheckPriorBounds(x):
        return -1e10 # big negative value, means zero basically if out of bounds
    
    rw, ri, rs, H, S = x
    
    # the first four are global vars; var_S is a variable parameter
    cov_x = np.diag([var_rw, var_ri, var_rs, var_H, var_S])
    
    return multivariate_normal.logpdf(x, mean=mu_x, cov=cov_x)

### Inverse Gamma Hyperprior on variances
Evaluates the log of the inverse-gamma density on some variance and hyperparameters $\alpha$ and $\beta$.

In [32]:
def LogInvGamma(var, alpha, beta):
    return invgamma.logpdf(var, a=alpha, scale=1/beta)

### Posterior on parameters
Evaluates log of the posterior on model parameters `x`:

$f(x|F,\epsilon,\sigma^2_S) \propto f(F | x, \epsilon ) \cdot f(x|\sigma^2_S)$

which is the likelihood times the prior on `x`.

In [34]:
def LogPosteriorX(x, F, var_eps, var_S):
    """
    ARGS:
        x (np.array): A numpy array of length 5 containing the model parameters: rw, ri, rs, H, S
        F (float): A scalar containing the observation of freeboard F
        var_eps (float): A scalar containing the given error variance
        var_S (float): A scalar containing the snow variance
        
    RETURNS:
        (float): A scalar containing log( f(x | F, var_eps, var_S) )
    """
    return LogLikelihood(x, F, var_eps) + LogPrior(x, var_S)

### Posterior on error
Evaluates log of the posterior on the error variance on 